# Homework 08
This homework is based on the clustering lectures. Check the lecture notes and TA notes - they should help!

## Question 1
This question will walk you through creating your own `kmeans` function.

#### a) What are the steps of `kmeans`?
**Hint**: There are 4 steps/builder functions that you'll need.

In [1]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
#a 
#1) Randomly assign points to clusters. 2) Compute cluster means. 3) Reassign points to the nearest centroid. 4) Recompute centroids.

#### b) Create the builder function for step 1.

In [3]:
#b
label_randomly <- function(n_points, n_clusters){
    sample(((1:n_points) %% n_clusters)+1, n_points, replace=F) #%% gives the remainder of points/clusters
}

#### c) Create the builder function for step 2.

In [4]:
#c 
get_cluster_means <- function(data, labels){
    data %>% 
      mutate(label__ = labels) %>% 
      group_by(label__) %>%
      summarize(across(where(is.numeric), mean)) %>%
      ungroup()
}

#### d) Create the builder function for step 3.
*Hint*: There are two ways to do this part - one is significantly more efficient than the other. You can do either.  

In [5]:
assign_cluster <- function(data, means) {
  data_num <- as.matrix(data %>% select(where(is.numeric)))
  means_num <- as.matrix(means %>% select(where(is.numeric), -label__))
  n_points <- nrow(data_num)
  n_clusters <- nrow(means_num)
  labels <- numeric(n_points)
  
  for (i in 1:n_points) {
    point <- data_num[i, ]
    dists <- colSums((t(means_num) - point)^2) # vectorized over clusters
    labels[i] <- means$label__[which.min(dists)]
  }
  labels
}

#### e) Create the builder function for step 4.

In [ ]:
#e
kmeans_done <- function(old_means, new_means, eps=1e-6){
    om <- as.matrix(old_means %>% select(where(is.numeric))) 
 
   nm <- as.matrix(new_means %>% select(where(is.numeric)))
    
    om <- om[order(old_means$label__), ]
    nm <- nm[order(new_means$label__), ]
    
    m <- mean(sqrt(rowSums((om - nm)^2)))
    return(m < eps)
}

#### f) Combine them all into your own `kmeans` function.

In [11]:
#f 
mykmeans <- function(data, n_clusters, eps=1e-6){
    labels <- label_randomly(nrow(data), n_clusters)
    old_means <- get_cluster_means(data, labels)
    done <- FALSE
    while (!done){
        labels <- assign_cluster(data, old_means)
        new_means <- get_cluster_means(data, labels)
        if (kmeans_done(old_means, new_means)){
            done <- TRUE
        } else {
            old_means <- new_means
        }
    }
    
    list(labels=labels, means=new_means)
}

## Question 2
This is when we'll test your `kmeans` function.
#### a) Read in the `voltages_df.csv` data set. 

In [12]:
#a 
voltage <- read.csv("voltages_df.csv")

#### b) Call your `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$means`. 

In [13]:
#b
results <- mykmeans(voltage, n_clusters = 3)
print(results$labels)
print(results$means)

  [1] 3 3 3 3 3 3 1 2 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 2 1 1 3 2 1 3 3
 [38] 3 1 3 3 3 3 2 3 2 2 3 2 2 3 1 3 3 1 3 3 3 1 3 3 3 3 3 2 2 3 1 3 2 3 3 1 1
 [75] 3 3 3 1 1 1 1 2 3 1 2 3 3 3 3 1 1 1 3 2 3 3 2 3 3 2 1 2 3 2 3 3 1 3 3 3 3
[112] 3 3 1 3 2 3 3 3 3 3 3 3 3 2 3 3 1 3 1 3 3 3 1 3 3 3 3 3 2 3 3 3 1 3 3 3 3
[149] 3 2 3 2 3 3 2 3 1 2 3 3 1 1 3 3 3 1 3 3 3 3 3 2 3 3 1 2 3 1 3 3 1 3 1 3 1
[186] 3 3 3 3 3 3 3 3 3 2 1 3 3 3 3 2 3 1 2 3 3 3 2 3 3 3 2 3 1 1 3 1 2 3 3 3 3
[223] 3 3 3 1 1 2 2 3 3 3 2 3 3 2 2 3 3 1 1 3 2 2 3 1 3 1 1 3 2 3 3 2 3 1 1 3 3
[260] 3 3 3 1 3 3 3 3 1 1 1 3 3 3 3 2 3 1 2 3 3 1 3 2 3 3 3 2 3 3 1 2 3 1 2 3 3
[297] 3 1 1 3 3 1 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 1 3 3 3 3 3 3 3 3 1
[334] 3 3 3 2 2 3 3 2 3 2 1 3 1 3 2 3 3 1 3 3 3 3 1 3 3 2 2 3 3 3 3 3 1 3 3 3 3
[371] 3 3 3 3 3 2 2 3 3 3 3 1 1 3 3 2 2 3 3 1 3 3 3 3 3 1 3 3 2 3 3 3 3 3 3 3 3
[408] 1 1 1 3 2 2 3 3 3 3 1 3 3 3 3 3 3 3 2 3 3 3 3 1 2 3 2 3 3 3 3 3 1 2 2 1 3
[445] 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 1 3 

#### c) Call R's `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$cluster`. 
*Hint*: Use the `as.matrix()` function to make the `voltages_df` data frame a matrix before calling `kmeans()`.

In [14]:
#c
voltages_matrix <- as.matrix(voltage %>% select(where(is.numeric)))
results <- kmeans(voltages_matrix, centers = 3)
print(results$cluster)
print(results$centers)

  [1] 1 3 3 3 3 3 2 2 3 1 1 1 2 1 3 1 1 1 1 1 3 3 3 3 3 1 1 1 2 2 2 2 3 2 2 3 3
 [38] 1 2 3 1 1 1 2 3 2 2 3 2 2 3 2 1 1 2 1 3 1 2 3 1 1 3 1 2 2 3 2 1 2 1 3 2 2
 [75] 3 3 1 2 2 2 2 2 3 2 2 3 3 3 1 2 2 2 3 2 3 3 2 3 3 2 2 2 3 2 1 1 2 1 1 3 1
[112] 1 3 2 1 2 1 1 3 3 3 3 1 1 2 3 3 2 1 2 3 1 3 2 1 3 3 3 1 2 1 3 3 2 1 1 3 1
[149] 1 2 1 2 3 3 2 1 2 2 3 1 2 2 1 3 1 2 3 1 3 1 1 2 3 1 2 2 3 2 1 3 2 3 2 1 2
[186] 3 3 1 3 1 1 3 1 1 2 2 1 3 1 1 2 1 2 2 3 1 3 2 3 1 3 2 1 2 2 3 2 2 3 3 3 3
[223] 3 3 3 2 2 2 2 1 3 1 2 3 3 2 2 1 1 2 2 1 2 2 3 2 1 2 2 3 2 3 1 2 3 2 2 1 1
[260] 1 3 3 2 1 3 3 3 2 2 2 1 3 3 3 2 1 2 2 3 3 2 1 2 1 1 3 2 3 3 2 2 1 2 2 1 3
[297] 1 2 2 3 3 2 1 1 1 2 1 3 3 3 1 1 1 1 1 1 3 1 3 1 1 2 3 2 1 1 1 1 3 1 3 1 2
[334] 1 3 3 2 2 1 3 2 3 2 2 1 2 3 2 1 1 2 3 3 3 1 2 3 1 2 2 3 3 3 3 1 2 1 3 1 1
[371] 1 1 1 3 3 2 2 1 1 1 1 2 2 1 1 2 2 3 3 2 1 3 3 3 3 2 3 3 2 1 1 1 1 3 3 1 1
[408] 2 2 2 3 2 2 1 3 3 3 2 3 3 3 1 1 3 1 2 3 3 3 3 2 2 3 2 1 3 1 1 1 2 2 2 2 1
[445] 3 1 1 1 2 3 1 1 1 3 3 1 3 3 1 2 3 

#### d) Are your labels/clusters the same? If not, why? Are your means the same?

In [3]:
#d The labels and clusters are not the same since cluster centers are chosen randomly. The means are similar but not the same. 

## Question 3
#### a) Explain the process of using a for loop to assign clusters for kmeans.

In [4]:
#a the loop allows to go through each point of the dataset, monitor the closest cluster found as it works its way through the dataset, check the distance from each point to each cluster center,and assign a point to the closest cluster

#### b) Explain the process of vectorizing the code to assign clusters for kmeans.

In [5]:
#b vecotrizing the code makes the kmeans function faster since it avoids the nested for loops. Instead of loop through each data point, the vectorization computes distances for all clusters at once

#### c) State which (for loops or vectorizing) is more efficient and why.

In [6]:
#c Vectorization is more efficient as it computes distances for all data points to all the clusters at one time instead of finding the distance for one data point before moving on to the next data point.

## Question 4
#### When does `kmeans` fail? What assumption does `kmeans` use that causes it to fail in this situation?

In [7]:
# kmeans fails when the assumptions are violated. these assumptions include spherical clusters, clusters of similar sizes that don't have a lot of overlap, and numeric and scaled features. In this situation
# the high dimensionality of the data is likely what caused the kmeans to fail.

## Question 5
#### What assumption do Guassian mixture models make?

In [8]:
# These models assume that data comes from a mix of mulitple normal distributions whose individual parameters are estimated from the data

## Question 6
#### What assumption does spectral clustering make? Why does this help us?

In [9]:
#Spectral clustering assumes that two points are more likely to be in the same cluster if they are close to one another. This is helpful because it allows us to work with more types of data

## Question 7
#### Define the gap statistic method. What do we use it for?

In [10]:
# The gap statistic method compares the clustering for each value of K to a cluster of data "randomized" into the same domain as the original data. The dispersion of the two clusterings is then calculated 
# and differences are observed. We look for a "knee" and that becomes the cluster number. Its used estimate the number of clusters based on the data.  